# Introduction
In this notebook, we are going to optimise the model to perform well on new (unseen) data.

Let's load in our data and import all the tools we will need.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
URL = 'https://github.com/data-analytics-in-business/lasagna-pipeline-demo/raw/main/data/sample_lasagna.csv'
df = pd.read_csv(URL)
df.head()

# Training and Testing
Let's specify our input $X$ and output $y$ variables, but let's also split them both into a set of samples we will use for training our model and a set of samples we will use for testing our model

In [ ]:
y = df['Have Tried']
X = df.drop(columns=['Person','Have Tried'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)


Given training and tesing data, we can create a classification pipeline like before, but this time we will train it on the training data and test it on the test data.

In [ ]:
numeric_features = ["Age", "Weight","Income","Car Value","CC Debt","Mall Trips"]
categorical_features = ["Pay Type","Gender","Live Alone","Dwell Type","Nbhd"]

preprocessor = ColumnTransformer(
    transformers=[
        ("num", MinMaxScaler(), numeric_features),
        ("cat", OneHotEncoder(drop='first', handle_unknown="ignore", sparse=False), categorical_features),
    ],
)

clf_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', DecisionTreeClassifier())
])

clf_pipeline.fit(X_train, y_train)
 
print('Training set score: ' + str(clf_pipeline.score(X_train,y_train)))
print('Test set score: ' + str(clf_pipeline.score(X_test,y_test)))

# Model Selection
We can use the idea of "training and testing" to optimise our model based on an estimate of its performance on unseen data.

To do this, we define a grid of parameters that we wish to search over. 

For this example, we will try different preprocessors for the numeric variables, and we will try different classifiers for the classifer "head" of our pipeline.

Run the code below to search over the parameter grid and print the results.

In [ ]:
param_grid = {
    'preprocessor__num' : [StandardScaler(), MinMaxScaler(), Normalizer()],
    'classifier' : [LogisticRegression(), DecisionTreeClassifier(), GaussianNB()]
}

grid = GridSearchCV(clf_pipeline, param_grid).fit(X_train, y_train)
 
df_results = pd.DataFrame(grid.cv_results_)
df_results[['param_classifier','param_preprocessor__num','mean_test_score','rank_test_score']].head(30)

**Question**: Which classification pipeline do you think will perform best on the unseen data?

# Exercise (Advanced)
Explore other [supervised learning](https://scikit-learn.org/stable/supervised_learning.html) classification methods and [preprocessing](https://scikit-learn.org/stable/modules/preprocessing.html) methods, and explore the parameters within those methods, to find the best `mean_test_score` you can find using the `sample_lasagna.csv` data.

In [ ]:
# (SOLUTION)